In [ ]:
import os
import pandas as pd
import faiss
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet50_Weights
from tqdm import tqdm

In [5]:
# Download dataset
!gdown 1w7Riq_itzcijqkvSW0yyivvlw0IgRGa_
!unzip Dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1w7Riq_itzcijqkvSW0yyivvlw0IgRGa_
To: /Users/86store/AIO2025/Module 2/Week 4/Dataset.zip
100%|██████████████████████████████████████| 1.34M/1.34M [00:00<00:00, 3.65MB/s]
Archive:  Dataset.zip
   creating: /Users/86store/AIO2025/Module 2/Week 4/Dataset
  inflating: Dataset/Avatar_Aaron_Eckhart.jpg  
  inflating: Dataset/Avatar_Aaron_Guiel.jpg  
  inflating: Dataset/Avatar_Amy_Pascal.jpg  
  inflating: Dataset/Avatar_Amy_Redford.jpg  
  inflating: Dataset/Avatar_Andrew_Bernard.jpg  
  inflating: Dataset/Avatar_Andrew_Cuomo.jpg  
  inflating: Dataset/Avatar_Anh_Khoi.JPG  
  inflating: Dataset/Avatar_Anil_Ramsook.jpg  
  inflating: Dataset/Avatar_Camille_Lewis.jpg  
  inflating: Dataset/Avatar_Carla_Gay_Balingit.jpg  
  inflating: Dataset/Avatar_Dang_Nha.jpg  
  inflating: Dataset/Avatar_Hoang_Nguyen.jpg  
  inflating: Dataset/Avatar_Minh_Chau.jpg  
  inflating: Dataset/Avatar_Phuc_Thinh.JPG  
  inflating: Dataset/Avatar_Quoc_Thai.JPG  
 

In [7]:
# Processing
dataset_path = 'Dataset'
os.listdir(dataset_path)
image_paths = []
labels = []
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.JPG', '.png', '.jpeg')):
        image_paths.append(os.path.join(dataset_path, filename))
        file_name = filename.split('.')[0]
        label = file_name[7:]
        labels.append(label)

In [9]:
df = pd.DataFrame({'image_path': image_paths, 'label': labels})

In [ ]:
## Vectorize Image
IMAGE_SIZE = 300
VECTOR_DIM = 300 * 300 * 3 # Vector Dimension for RGB images

In [13]:
# Create FAISS Index
index = faiss.IndexFlatL2(VECTOR_DIM)
label_map = []

In [15]:
def image_to_vector(image_path):
    """Convert image to normalized vector"""
    img = Image.open(image_path).resize((IMAGE_SIZE, IMAGE_SIZE))
    img_array = np.array(img)

    # Handle grayscale images (convert to RGB)
    if len(img_array.shape) == 2:
        img_array = np.stack((img_array,)*3, axis=-1)

    # Normalize pixel values to [0, 1]
    vector = img_array.astype('float32') / 255.0
    return vector.flatten()

In [17]:
# Xây dựng index từ dataframe
for idx, row in df.iterrows():
    # Get image_path, label from dataframe
    image_path = row['image_path']
    label = row['label']

    try:
        # Convert each imape_path to vector by image_to_vector function
        vector = image_to_vector(image_path)

        # Add to Faiss 
        index.add(np.array([vector]))
        label_map.append(label)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

In [19]:
# Save the index and label map for later use
faiss.write_index(index, "employee_images.index")
np.save("label_map.npy", np.array(label_map))

In [21]:
def search_similar_images(query_image_path, k=5):
    """Search for similar employee images"""
    # Load index and labels
    index = faiss.read_index("employee_images.index")
    label_map = np.load("label_map.npy")

    # Convert query image to vector
    query_vector = image_to_vector(query_image_path)

    # Search in Faiss
    distances, indices = index.search(np.array([query_vector]), k)

    # Get result
    result = []
    for i in range(len(indices[0])):
        employee_name = label_map[indices[0][i]]
        distance = distances[0][i]
        result.append((employee_name, distance))

    return result

In [23]:
def display_query_and_top_matches(query_image_path):
    query_img = Image.open(query_image_path)
    query_img = query_img.resize((300, 300))

    plt.figure(figsize=(5, 5))
    plt.imshow(query_img)
    plt.title("Query Image")
    plt.axis('off')
    plt.show()

    matches = search_similar_images(query_image_path)

    """
    Display the top 5 matching employee images with distances
    """
    # Get the image paths for the result
    top_matches = []
    for name, distance in matches:
        # Find the image path for this employee in df
        img_path = df[df['label'] == name]['image_path'].values[0]
        top_matches.append((name, distance, img_path))
    top_matches

    # Create plot
    plt.figure(figsize=(15, 15))
    for i, (name, distance, img_path) in enumerate(top_matches):
        img = Image.open(img_path)
        img = img.resize((300, 300))

        plt.subplot(1, 5, i+1)
        plt.imshow()
        plt.title(f"{name}\nDist: {distance:.2f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [25]:
# Initialize ResNet50 model for feature extraction
weights = ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)

# Remove the last classification layer
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Set model to evaluation mode
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [28]:
# Install Pretrained model (for face recognition task)
from facenet_pytorch import InceptionResnetV1
face_recognition_model = InceptionResnetV1(pretrained='vggface2').eval()

In [31]:
# Resize
transform = transforms.Compose([
    transforms.Resize((300, 300)), # Resize to the model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [33]:
def extract_feature(image_path, model):
    """Extract features from an image using a given model"""
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = model(img_tensor)
    return features.squeeze().numpy()

In [35]:
# Create FAISS index
VECTOR_DIM = 512
index = faiss.IndexFlatIP(VECTOR_DIM)
label_map = []

# Add feature vector to index
for idx, row in tqdm(df.iterrows(), total=len(df)):
    features = extract_feature(row['image_path'], face_recognition_model)
    index.add(np.array([features]))
    label_map.append(row['label'])

100%|███████████████████████████████████████████| 18/18 [00:03<00:00,  4.56it/s]


In [37]:
# Save index and labels
faiss.write_index(index, "facenet_features.index")
np.save("facenet_label_map.npy", np.array(label_map))

In [52]:
def image_to_feature(image_path, model):
    """Convert image to face embedding using a pre-trained model"""
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0) # Add batch dimension

    # Get the embedding
    with torch.no_grad(): # Disable gradient calculation
        embedding = model(img_tensor)

    # Return the embedding as a numpy array
    return embedding.squeeze().numpy()

In [54]:
def search_similar_images(query_image_path, k=5):
    """Search for similar employee images using VGG16 features"""
    # Load index and labels
    index = faiss.read_index("facenet_features.index")
    label_map = np.load("facenet_label_map.npy")

    # Convert query image to vector
    query_vector = image_to_feature(query_image_path, face_recognition_model)

    # Search in Faiss
    similarities, indices = index.search(np.array([query_vector]), k)

    # Get results
    results = []
    for i in range(len(indices[0])):
        employee_name = label_map[indices[0][i]]
        similarity = similarities[0][i]
        results.append((employee_name, similarity))

    return results

In [56]:
def display_query_and_top_matches(query_image_path):
    # Display query image
    query_img = Image.open(query_image_path)
    query_img = query_img.resize((300, 300))
    plt.figure(figsize=(5, 5))
    plt.imshow(query_img)
    plt.title("Query Image")
    plt.axis('off')
    plt.show()

    # Get matches
    matches = search_similar_images(query_image_path)

    # Display top matches
    plt.figure(figsize=(15, 5))
    for i, (name, similarity) in enumerate(matches):
        # Find the image path for this employee
        img_path = df[df['label'] == name]['image_path'].values[0]
        img = Image.open(img_path)
        img = img.resize((300, 300))

        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f"{name}\nSimilarity: {similarity:.2f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()